In [1]:
import psycopg2
import pandas as pd
import datacompy
import pyodbc
import useful_functions as use

In [3]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [4]:
business_sql = '''
    SELECT * FROM ONEVIEW.BUSINESS
    '''
ov_business_aff = pd.read_sql_query(business_sql, conn)

In [5]:
provider_sql = '''
    SELECT * FROM ONEVIEW.PROVIDER_AFFILIATION
    '''
ov_provider_aff = pd.read_sql_query(provider_sql, conn)
ov_provider_aff

,id,business,medical_education_number,type,description,primary,rank,group,group_description,batch_business_date
0,INS000056380397908026,INS00005638,03979080262,1,Attending,A,8,ATT,Attending,2021-04-23
1,INS000055350287816041,INS00005535,02878160412,1,Attending,P,9,ATT,Attending,2021-04-23
2,INS000066410411212166,INS00006641,04112121663,1,Attending,A,8,ATT,Attending,2021-04-23
3,INS000051000411503138,INS00005100,04115031380,1,Attending,P,10,ATT,Attending,2021-04-23
4,INS000032850251217193,INS00003285,02512171934,1,Attending,P,9,ATT,Attending,2021-04-23
...,...,...,...,...,...,...,...,...,...,...
1528605,INS000034560384005039,INS00003456,03840050393,3,Admitting,A,6,ADM,Admitting,2021-04-23
1528606,INS000036538471075003,INS00003653,84710750032,3,Admitting,A,6,ADM,Admitting,2021-04-23
1528607,INS000029019570203001,INS00002901,95702030014,3,Admitting,A,6,ADM,Admitting,2021-04-23
1528608,INS000013347481197001,INS00001334,74811970011,3,Admitting,A,6,ADM,Admitting,2021-04-23


In [7]:
ppd_sql = '''
    SELECT * FROM ONEVIEW.PHYSICIAN
    '''
ov_ppd = pd.read_sql_query(ppd_sql, conn)

In [8]:
username = 'vigrose'
password = 'Ravenclaw~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)

In [70]:
iqvia_providers = \
'''
SELECT *
FROM ODS.ODS_IMS_PROFESSIONAL P
'''
providers = pd.read_sql(con=ODS, sql=iqvia_providers)
providers.head()

,HCE_ID_PRFSNL,PROFESSIONAL_ID,PARTY_ID,FIRST_NAME,MIDDLE_NAME,LAST_NAME,GEN_SUFFIX,DESIGNATION,GENDER,ROLE,...,PRIMARY_PROF_DESC,IMS_ID,UPIN,ME,ALL_DEA,NPI,STATUS_DESC,JOB_ID,CURRENT_BATCH_FLAG,BATCH_BUSINESS_DATE
0,6909898,2,2800318.0,ARNOLD,H,MENEZES,None,None,M,P,...,Medical Doctors,0141528,A00005,4951767005,None,1881687903,Active,94043,Y,2021-04-23
1,6906117,10,2044792.0,ANDREW,CLIFFORD,SMITH,None,None,M,B,...,Medical Doctors,0141477,None,0180380133,None,1568451367,Active,94043,Y,2021-04-23
2,7721605,18,2606213.0,JEAN,E,ROBILLARD,None,None,M,P,...,Medical Doctors,1735358,A00077,0670268005,None,1033108170,Active,94043,Y,2021-04-23
3,7032674,26,1957487.0,JOHN,HENRY,BRINKMAN,None,None,M,P,...,Medical Doctors,0401909,A00113,0180362016,None,1760599864,Active,94043,Y,2021-04-23
4,6971199,34,2109542.0,DALE,HOWARD,WEBER,None,None,M,B,...,Medical Doctors,0214608,A00142,0180364093,None,1205880671,Active,94043,Y,2021-04-23


In [5]:
iqvia_query = \
        """
        SELECT distinct
        P.ME,
        P.PROFESSIONAL_ID,
        A.AFFIL_IND as primary,
        A.AFFIL_RANK as rank,
        A.AFFIL_TYPE_ID as type,
        A.IMS_ORG_ID as business,
        T.AFFIL_TYPE_DESC as description,
        T.AFFIL_GROUP_CODE as group,
        T.AFFIL_GROUP_DESC as group_description,
        A.BATCH_BUSINESS_DATE
        FROM 
        ODS.ODS_IMS_PROFESSIONAL P, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        P.ME IS NOT NULL
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,ME,PROFESSIONAL_ID,PRIMARY,RANK,TYPE,BUSINESS,DESCRIPTION,GROUP,GROUP_DESCRIPTION,BATCH_BUSINESS_DATE
0,4951767005,2,P,10,1,INS00006505,Attending,ATT,Attending,2021-04-23
1,4951767005,2,P,10,8,INS00504324,Admitting (Inferred),ADM,Admitting,2021-04-23
2,0180380133,10,P,10,1,INS00005054,Attending,ATT,Attending,2021-04-23
3,0180380133,10,A,6,3,INS00005339,Admitting,ADM,Admitting,2021-04-23
4,0180380133,10,A,6,5,INS00076298,Consulting,LTC,Long Term Care,2021-04-23


In [12]:
iqvia['id'] = iqvia.business + iqvia.me
iqvia.columns = [c.lower() for c in iqvia.columns.values]
ov_ppd['ME'] = [x[:-1] for x in ov_ppd.medical_education_number]
PPD = ov_ppd[['ME', 'medical_education_number']]
iqvia['group'] = [x.strip() for x in iqvia.group]
affiliations = pd.merge(iqvia, PPD, left_on='me', right_on='ME')

In [80]:
affiliation_compare = datacompy.Compare(ov_provider_aff, affiliations[ov_provider_aff.columns], join_columns='id', df1_name = 'oneview', df2_name = 'iqvia')

In [81]:
print(affiliation_compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns     Rows
0   oneview       10  1528911
1     iqvia       10  1528911

Column Summary
--------------

Number of columns in common: 10
Number of columns in oneview but not in iqvia: 0
Number of columns in iqvia but not in oneview: 0

Row Summary
-----------

Matched on: id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 1,528,911
Number of rows in oneview but not in iqvia: 0
Number of rows in iqvia but not in oneview: 0

Number of rows with some compared columns unequal: 1,528,911
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 9
Total number of values which compare unequal: 1,528,911

Columns with Unequal Values or Types
------------------------------------

  Column oneview dtype iqvia dt